In [ ]:
%load_ext autoreload
%autoreload 2
import sys
import os
ProjDIR = "/home/jw3514/Work/ASD_Circuits_CellType/" # Change to your project directory
sys.path.insert(1, f'{ProjDIR}/src/')
from ASD_Circuits import *

try:
    os.chdir(f"{ProjDIR}/notebook_rebuttal/")
    print(f"Current working directory: {os.getcwd()}")
except FileNotFoundError as e:
    print(f"Error: Could not change directory - {e}")
except Exception as e:
    print(f"Unexpected error: {e}")

HGNC, ENSID2Entrez, GeneSymbol2Entrez, Entrez2Symbol = LoadGeneINFO()

In [ ]:
GeneExpCorr = pd.read_csv("../dat/ISH_MERFISH_Gene_CorssSTR_Corr.v3.csv", index_col=0)

In [ ]:
import glob

GW_DIR = ProjDIR + "dat/Genetics/GeneWeights/"
GW_DN_DIR = ProjDIR + "dat/Genetics/GeneWeights_DN/"

# Ensure output directory exists
os.makedirs(GW_DN_DIR, exist_ok=True)

# List all files ending with .gw in GW_DIR
gw_files = glob.glob(os.path.join(GW_DIR, "*.gw"))

print(f"Found {len(gw_files)} '.gw' files in {GW_DIR}:")
for f in gw_files:
    print(" -", os.path.basename(f))

for gw_file in gw_files:
    gw_basename = os.path.basename(gw_file)
    # Change filename to end with .DN.gw
    if gw_basename.endswith(".gw"):
        output_basename = gw_basename[:-3] + ".DN.gw"
    else:
        output_basename = gw_basename + ".DN.gw"  # fallback
    
    # Load gene weights dictionary
    gene_weights = Fil2Dict(gw_file)
    gene_weights_adj = {}
    for k, v in gene_weights.items():
        if k in GeneExpCorr.index.values:
            gene_weights_adj[k] = v * (GeneExpCorr.loc[k, "V2_V3_CT_Corr"] ** 2)
    # Write the adjusted gene weights to output directory, with new filename
    output_file = os.path.join(GW_DN_DIR, output_basename)
    Dict2Fil(gene_weights_adj, output_file)
    print(f"Adjusted weights written to: {output_file}")